# Model training

## Settings

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import helpers.settings as sts
sts.print_settings(sts)

BEST_ESTIMATOR_FILENAME : best_estimator_0.0.1.pkl
DATASET_TRAIN_FILENAME : dataset_train.parquet
DATASET_VALIDATION_FILENAME : dataset_validation.parquet
ETL_VERSION : 0.0.1
MODEL_FILENAME : model.pkl
MODEL_VERSION : 0.0.1
PREPROCESSOR_FILENAME : preprocessor_0.0.1.pkl
TRAINED_BEST_ESTIMATOR_FILENAME : trained_best_estimator_0.0.1.pkl
color : <class 'helpers.settings.color'>
print_settings : <function print_settings at 0x7fb2489afc10>



## Imports

In [4]:
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics import (
    classification_report,
    plot_precision_recall_curve,
    precision_recall_curve,
    average_precision_score,
)
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint as sp_randint
import shap

## Load train dataset

In [5]:
df_train = pd.read_parquet(f"data/{sts.DATASET_TRAIN_FILENAME}")

In [6]:
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:,-1]

## Best estimator param grid

In [7]:
estimators = [
    {
        "clf": [RandomForestClassifier()],
        "clf__bootstrap": [True, False],
        "clf__max_depth": [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
        "clf__max_features": ["auto", "sqrt"],
        "clf__min_samples_leaf": [1, 2, 4],
        "clf__min_samples_split": [2, 5, 10],
        "clf__n_estimators": [100, 200, 400, 500]
    },
    {
        "clf": [XGBClassifier()],
        "clf__objective": ["binary:logistic"],
        "clf__use_label_encoder": [False],
        "clf__eval_metric": ["logloss"],
        "clf__learning_rate": [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3],
        "clf__min_child_weight": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        "clf__gamma": [0.5, 1, 1.5, 2, 5],
        "clf__subsample": np.random.uniform(1, .7, 1),
        "clf__colsample_bytree": [ 0.3, 0.4, 0.5 , 0.7 ],
        "clf__max_depth": [3, 4, 5, 6, 7, 8],
        "clf__n_estimators": np.arange(100, 500, 10),
        
    },
    {
        "clf": [LGBMClassifier()],
        "clf__num_leaves": sp_randint(6, 50), 
        "clf__min_child_samples": sp_randint(100, 500), 
        "clf__min_child_weight": [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
        "clf__subsample": np.random.uniform(0.2, 0.8, 1), 
        "clf__colsample_bytree": np.random.uniform(0.4, 0.6, 1),
        "clf__reg_alpha": [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
        "clf__reg_lambda": [0, 1e-1, 1, 5, 10, 20, 50, 100],
    },  
]

## Load best estimator

In [8]:
with open(f"artifacts/{sts.BEST_ESTIMATOR_FILENAME}", "rb") as file:
    best_estimator = pickle.load(file)

In [9]:
rs = RandomizedSearchCV(
    best_estimator, 
    estimators, 
    cv=3,
    scoring="average_precision",
    n_jobs=1, 
    verbose=3, 
    n_iter=20, 
    random_state=42,
)

In [10]:
rs.fit(X_train, y_train)

2021-09-05 18:41:50.287887 INFO: Hard mode for the ModifiedColumnTransformer set to True: The initial features are going to be enforced during transformation and fit steps
2021-09-05 18:41:50.294727 INFO: Hard mode for the ModifiedColumnTransformer set to True: The initial features are going to be enforced during transformation and fit steps
is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead


Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV 1/3] END clf=LGBMClassifier(), clf__colsample_bytree=0.43625487472410873, clf__min_child_samples=448, clf__min_child_weight=1000.0, clf__num_leaves=26, clf__reg_alpha=10, clf__reg_lambda=0.1, clf__subsample=0.7009470777838764;, score=0.013 total time=   0.4s


2021-09-05 18:41:50.690356 INFO: Hard mode for the ModifiedColumnTransformer set to True: The initial features are going to be enforced during transformation and fit steps
is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead


[CV 2/3] END clf=LGBMClassifier(), clf__colsample_bytree=0.43625487472410873, clf__min_child_samples=448, clf__min_child_weight=1000.0, clf__num_leaves=26, clf__reg_alpha=10, clf__reg_lambda=0.1, clf__subsample=0.7009470777838764;, score=0.013 total time=   0.4s


2021-09-05 18:41:51.106199 INFO: Hard mode for the ModifiedColumnTransformer set to True: The initial features are going to be enforced during transformation and fit steps
is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead


...

[CV 2/3] END clf=XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None), clf__colsample_bytree=0.5, clf__eval_metric=logloss, clf__gamma=0.5, clf__learning_rate=0.2, clf__max_depth=4, clf__min_child_weight=7, clf__n_estimators=230, clf__objective=binary:logistic, clf__subsample=0.7704077484298268, clf__use_label_encoder=False;, score=0.540 total time=   1.2s


2021-09-05 18:44:00.647905 INFO: Hard mode for the ModifiedColumnTransformer set to True: The initial features are going to be enforced during transformation and fit steps
is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead


[CV 3/3] END clf=XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None), clf__colsample_bytree=0.5, clf__eval_metric=logloss, clf__gamma=0.5, clf__learning_rate=0.2, clf__max_depth=4, clf__min_child_weight=7, clf__n_estimators=230, clf__objective=binary:logistic, clf__subsample=0.7704077484298268, clf__use_label_encoder=False;, score=0.586 total time=   1.2s


2021-09-05 18:44:01.817928 INFO: Hard mode for the ModifiedColumnTransformer set to True: The initial features are going to be enforced during transformation and fit steps
2021-09-05 18:44:01.819679 INFO: Hard mode for the ModifiedColumnTransformer set to True: The initial features are going to be enforced during transformation and fit steps
is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ModifiedColumnTransformer(categorical_features=['CODE_GENDER',
                                                                                              'FLAG_OWN_CAR',
                                                                                              'FLAG_OWN_REALTY',
                                                                                              'NAME_INCOME_TYPE',
                                                                                              'NAME_EDUCATION_TYPE',
                                                                                              'NAME_FAMILY_STATUS',
                                                                                              'NAME_HOUSING_TYPE',
                                                                                              'FLAG_MOBIL',
           

In [11]:
pd.DataFrame(rs.cv_results_).query("rank_test_score == 1")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_clf__colsample_bytree,param_clf__min_child_samples,param_clf__min_child_weight,param_clf__num_leaves,param_clf__reg_alpha,...,param_clf__max_features,param_clf__min_samples_leaf,param_clf__min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
9,6.925394,0.078539,0.461552,0.001079,"RandomForestClassifier(bootstrap=False, max_de...",NaN,NaN,NaN,NaN,NaN,...,sqrt,1,5,{'clf': RandomForestClassifier(bootstrap=False...,0.889026,0.811945,0.868832,0.856601,0.032635,1


## Dump trained best estimator

In [12]:
with open(f"artifacts/{sts.TRAINED_BEST_ESTIMATOR_FILENAME}","wb") as file:
    pickle.dump(rs.best_estimator_, file)